In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP),
    RobotConfig(1, RobotType.panda, None,
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## init stereo aruco detector

In [3]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.83028054, -0.03017845, -0.55652803,  0.6479348 ],
        [ 0.17372349,  0.9628003 ,  0.20696786, -0.1428074 ],
        [ 0.5295794 , -0.2685234 ,  0.8046371 ,  0.07192612],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## init scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo, base_link="base_link")
s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## add environment

In [6]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

Please create a subscriber to the marker


In [7]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]})

In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add movable

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [10]:
goal = gscene.NAME_DICT["goal"]
floor = gscene.NAME_DICT['floor']
floor.color = floor.color[:3]+(0.5,)
goal.set_offset_tf(center=tuple(np.matmul(floor.orientation_mat, (-0.5,-0.2,0))), 
                   orientation_mat=floor.orientation_mat)
gscene.set_rviz()

## add brush

In [11]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(-0.7,-0.2,0.08), rpy=(0,0,np.pi/2), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), center=(0,0,-0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_handle")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [12]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [13]:
from pkg.planning.constraint.constraint_action import Gripper2Tool, PlacePlane

In [14]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [15]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))

## add objects

In [16]:
from pkg.planning.constraint.constraint_object import BoxAction, CustomObject, Grasp2Point, PlacePoint

In [17]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxAction, hexahedral=True)
box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxAction, hexahedral=True)

In [18]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush = pscene.create_object(oname="brush", gname="brush_handle", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]), 
                                                   "face": PlacePoint("face", brush_handle, [0,0,-0.075], [0,0,0])})

## Register object binders

In [19]:
box1.register_binders(pscene, PlacePlane)
box2.register_binders(pscene, PlacePlane)

### planning pipeline

In [20]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.object_a_star import ObjectAstar
tplan = ObjectAstar(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## ui

In [21]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Test plan

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

obj, handle, binder = "brush", "face", "goal_bd"
initial_state = pscene.get_state(crob.home_pose)
goal_nodes = pscene.get_goal_nodes(initial_state.node, obj, binder)
ppline.search(initial_state, goal_nodes, verbose=True, display=True, dt_vis=0.01)
schedules = ppline.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
Not available:brush-grip1
np_exclude:handle
bd_exclude:grip0
============== Non-available transition: Break =====================
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = success
Remaining:2->1 / branching: 0->1 (1.09/600.0 s, steps/err: 12(406.646966934 ms)/1.82970587357)
Not available:brush-grip1
np_exclude:handle
bd_exclude:grip0
============== Non-available transition: Break =====================
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = success
Remaining:2->1 / branching: 0->2 (1.55/600.0 s, steps/err: 12(460.304021835 ms)/1.77086284392)
node: ('floor', 'floor', 'floor')->('floor', 'floor', 

node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip1') = fail
node: ('floor', 'flo

In [23]:
schedule = ppline.idxSchedule2SnodeScedule(ppline.find_schedules().values()[0])

In [24]:
ppline.play_schedule(schedule)

In [25]:
print(schedule[0].state.node)
print(schedule[1].state.node)
print(schedule[2].state.node)

(('box1', 'back_p', 'floor'), ('box2', 'back_p', 'floor'), ('brush', 'face', 'floor'))
(('box1', 'back_p', 'floor'), ('box2', 'back_p', 'floor'), ('brush', 'handle', 'grip0'))
(('box1', 'back_p', 'floor'), ('box2', 'back_p', 'floor'), ('brush', 'face', 'floor'))


In [26]:
idx = 0
from_state = schedule[idx].state
to_state = schedule[idx+1].state
redundancy_dict = schedule[idx+1].redundancy_dict
redundancy_dict[obj]['w'] = 0
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)
if success:
    for bd in binding_list:
        pscene.rebind(bd, list2dict(lastQ, pscene.gscene.joint_names))

True


In [27]:
new_state = to_state.copy(pscene)
new_state.obj_pos_dict = pscene.get_object_state()[1]
new_state.Q = lastQ

In [28]:
gscene.show_motion(traj)

In [29]:
idx = 1
from_state = new_state.copy(pscene)
to_state = from_state.copy(pscene)
to_state.node = (('box1', 'back_p', 'floor'),
            ('box2', 'back_p', 'floor'),
            ('brush', 'face', 'goal_bd'))
redundancy_dict = schedule[idx+1].redundancy_dict
redundancy_dict = {obj:{'w': np.pi/2}}
traj, lastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, 
                                                    redundancy_dict=redundancy_dict)
print(success)

True


In [30]:
gscene.show_motion(traj, period=0.05)

In [31]:
from pkg.planning.motion.moveit.moveit_planner import make_constraint_list

##
# @class MotionManifold
# @brief geometry manifold to constrain motion
class MotionManifold:
    ##
    # @param geometry_list list of rnb-planning.src.pkg.geometry.geometry.GeometryItem
    def __init__(self, geometry_list):
        self.geometry_list = geometry_list
        
        
##
# @brief add motion constraint for moveit planner
# @param
def add_constraint(self, group_name, tool_link, tool_offset_T, manifold, fix_surface, fix_normal, tol=1e-3, use_box=False):
    constraint_manifold_list = make_constraint_list(manifold.geometry_list, use_box=use_box)
    xyzquat = T2xyzquat(tool_offset_T)
    self.planner.add_union_manifold_py(group_name=group_name, tool_link=tool_link, tool_offset=xyzquat[0]+xyzquat[1], 
                                       geometry_list=constraint_manifold_list, fix_surface=fix_surface, fix_normal=fix_normal, tol=tol)

In [32]:
from pkg.planning.constraint.constraint_common import calc_redundancy
self = mplan
if len(binding_list)!=1:
    raise(RuntimeError("Only single manipulator operation is implemented with moveit!"))
self.update_gscene()

obj_name, ap_name, binder_name = binding_list[0]
redundancy = redundancy_dict[obj_name] if redundancy_dict else None

binder = self.pscene.binder_dict[binder_name]
obj = self.pscene.object_dict[obj_name]
handle = obj.action_points_dict[ap_name]
point_add, rpy_add = calc_redundancy(redundancy, binder)
T_handle = handle.Toff_lh
T_binder = np.matmul(binder.Toff_lh, SE3(Rot_rpy(rpy_add), point_add))

if len(self.planner.group_names)==1:
    group_name_handle = self.planner.group_names if handle.geometry.link_name in self.urdf_content.parent_map else []
    group_name_binder = self.planner.group_names if binder.geometry.link_name in self.urdf_content.parent_map else []
else:
    group_name_handle = [gname for gname in self.planner.group_names if gname in handle.geometry.link_name]
    group_name_binder = [gname for gname in self.planner.group_names if gname in binder.geometry.link_name]

dual = False
if group_name_binder and not group_name_handle:
    group_name = group_name_binder[0]
    tool, T_tool = binder, T_binder
    target, T_tar = handle, T_handle
elif group_name_handle and not group_name_binder:
    group_name = group_name_handle[0]
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder
else:
    if not self.enable_dual:
        raise(RuntimeError("dual arm motion is not enabled"))
    dual = True
    group_name = "{}_{}".format(group_name_binder[0], group_name_handle[0])
    self.update_gscene(group_name)
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder

T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
goal_pose = tuple(T_tar_tool[:3,3]) \
            +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())

if dual:
    dual_planner = self.dual_planner_dict[group_name]
    planner = dual_planner.planner
    from_Q = from_state.Q[dual_planner.idx_pscene_to_mpc]*dual_planner.joint_signs
else:
    planner = self.planner
    if self.need_mapping:
        from_Q = from_state.Q[self.idx_pscene_to_mpc]
    else:
        from_Q = from_state.Q

In [33]:
# manifold = MotionManifold([gscene.NAME_DICT["floor"]])

In [34]:
gscene.show_pose(from_Q)

In [35]:
floor = gscene.NAME_DICT["floor"]
floor_top = gscene.copy_from(floor, new_name="floor_top", color=(0,1,0,0.8), display=True, collision=False)
floor_top.set_offset_tf(center=tuple(np.add(floor_top.center, (0,0,floor_top.dims[2]/2))), 
                        orientation_mat=np.matmul(floor_top.orientation_mat, Rot_axis(1,0)))
floor_top.set_dims(floor_top.dims[:2]+(0.001,))

In [36]:
T_goal = T_xyzquat((goal_pose[:3], goal_pose[3:]))
goal_pose_marker = gscene.create_safe(name="goal_pose", gtype=GEOTYPE.SPHERE, link_name="base_link",
                   dims=(0.01, 0.01, 0.01), center=tuple(T_goal[:3,3]), rpy=Rot2rpy(T_goal[:3,:3]), collision=False, display=True,
                   color=(1,0,0,1))

In [37]:
gscene.set_rviz()

In [38]:
gscene.add_highlight_axis("ax", "goal_pose", goal_pose_marker.link_name, goal_pose_marker.center, goal_pose_marker.orientation_mat)

In [39]:
gscene.show_pose(lastQ)
print(lastQ)

[ 0.3852608  -0.45200295 -1.98997688  0.00271417 -0.69100489  0.38262492
  0.         -0.39269908  0.         -1.57079633  0.          1.57079633
  1.57079633]


In [40]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
manifold = MotionManifold([floor_top])
add_constraint(mplan, group_name, tool.geometry.link_name, tool.Toff_lh, manifold, fix_surface=True, fix_normal=True, tol=2e-3)

In [41]:
trajectory, success = mplan.planner.plan_constrained_py(
    group_name,tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=10)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (27)


In [42]:
gscene.show_motion(trajectory, period=0.05)

### eTaSL

In [22]:
from pkg.planning.motion.etasl.etasl import EtaslPlanner
eplan = EtaslPlanner(pscene)
eplan.update_gscene()

In [24]:
traj, lastQ, error, success, binding_list = eplan.plan_transition(from_state, to_state, 
                                                    N=1000)
print(success)

True


In [25]:
gscene.show_motion(traj, period=0.05)

## Disconnect stereo

In [26]:
stereo.disconnnect()